<a href="https://colab.research.google.com/github/anuragsaraf1912/Babbar450/blob/master/getMarketData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

In [54]:
def plotCandleStick(data):
  """Plots candlestick pattern using the Plotly library"""
  fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                      open=data['Open'],
                                      high=data['High'],
                                      low=data['Low'],
                                      close=data['Close'],
                                      increasing_line_color='green',  # Color for increasing candles
                                      decreasing_line_color='red'     # Color for decreasing candles
                                      )])

  # Update layout
  fig.update_layout(title='Candlestick Chart',
                    xaxis_title='Date',
                    yaxis_title='Price')

  # Show chart
  fig.show()

In [55]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        startDate = start_date_picker.value
        endDate = end_date_picker.value
        ticker = text_entry.value + ".NS"
        if startDate and endDate and ticker:
          df = yf.download(ticker, startDate, endDate)
          plotCandleStick(df.reset_index())


# Date Picker Widgets
start_date_picker = widgets.DatePicker(description='Start Date:')
end_date_picker = widgets.DatePicker(description='End Date:')

# Text Entry Widget
text_entry = widgets.Text(description='Stock Ticker:')

# Observing changes
start_date_picker.observe(on_value_change)
end_date_picker.observe(on_value_change)
text_entry.observe(on_value_change)

# Displaying widgets
widgets.HBox([text_entry, start_date_picker, end_date_picker])

#Some stocks to look into: HDFCBANK, WSTCSTPAPR

[*********************100%%**********************]  1 of 1 completed


In [51]:
fig

In [31]:
len(text_entry.value) and True

0